In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

import pandas as pd
import numpy as np

import time

import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
os.makedirs('results/kpriors', exist_ok=True)

In [5]:
use_cuda = True

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L71
learning_rate = 0.005

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L20
adaptation_task = 'remove_data'

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L83
prior_prec = 5

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L31
adaptation_methods = ['Replay','K-priors']

In [6]:
sys.path.append(os.path.abspath('./libraries/kpriors/'))

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L114
from adamreg import AdamReg
import utils
import train

In [7]:
# https://github.com/team-approx-bayes/kpriors/blob/main/models.py#L75

# Return all parameters as a vector
def return_parameters(model):
    num_params = sum([np.prod(p.size()) for p in model.parameters()])
    means = torch.zeros(num_params)

    start_ind = 0
    for p in model.parameters():
        num = np.prod(p.size())
        means[start_ind:start_ind+num] = p.data.reshape(-1)
        start_ind += num

    return means

In [10]:
def fit(base_model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/team-approx-bayes/kpriors/blob/main/train.py#L10
    error = torch.nn.CrossEntropyLoss()
    
    # prepare model
    
    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L159
    model = copy.deepcopy(base_model)
    optimiser = AdamReg(model, lr=learning_rate, weight_decay=prior_prec)
    optimiser.previous_weights = return_parameters(base_model)

    def select_memory_points(base_train_data, additional_memory_data):
        
        # Select points
        # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L144
        memory_points = {}
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L63
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L72
        memory_points['inputs'] = torch.cat((base_train_data[0], additional_memory_data[0]))
        memory_points['true_labels'] = torch.cat((base_train_data[1], additional_memory_data[1]))
        if use_cuda:
            memory_points['inputs'] = memory_points['inputs'].cuda()
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L79
        memory_points['soft_labels'] = torch.softmax(base_model.forward(memory_points['inputs']), dim=-1)
        
        
        # Soft labels in K-priors, hard (true) labels in Replay
        if adaptation_method == "K-priors":
            memory_points['labels'] = memory_points['soft_labels']
        elif adaptation_method == "Replay":
            memory_points['labels'] = torch.nn.functional.one_hot(memory_points['true_labels'], num_classes=10)
        
        # Store past memory labels
        # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L230
        optimiser.memory_labels = memory_points['labels']

        return memory_points

    
    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L218
    optimiser.prior_prec_old = prior_prec
    remove_data_bool = True

    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L49
    train_batch_size = int(np.ceil(BATCH_SIZE * len(train_set) / (len(train_set) + len(forget_set))))
    forget_batch_size = int(np.ceil(BATCH_SIZE * len(forget_set) / (len(train_set) + len(forget_set))))

    num_steps = min(len(train_set) // train_batch_size, len(forget_set) // forget_batch_size)

    train_x, train_y = train_set.tensors[0], train_set.tensors[1]
    forget_x, forget_y = forget_set.tensors[0], forget_set.tensors[1]
    
    train_times = list()
    
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()

        model.train()
        for i in range(num_steps):
            
            # If remove_data task, then store the removed points too, for both K-priors and Replay
            
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L140
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L134
            base_train_data = (train_x[train_batch_size*i:train_batch_size*(i+1)], train_y[train_batch_size*i:train_batch_size*(i+1)])
            additional_memory_data = (forget_x[forget_batch_size*i:forget_batch_size*(i+1)], forget_y[forget_batch_size*i:forget_batch_size*(i+1)])
            
            # Load data for adaptation task
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L148
            # https://github.com/team-approx-bayes/kpriors/blob/main/data_generators.py#L67
            adapt_train_data = additional_memory_data
    
            memory_points = select_memory_points(base_train_data, additional_memory_data)
                
            # Train model
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L235
            
            train.train_model(
                model, optimiser, adapt_train_data, 
                num_epochs=1, # one epoch
                memory_data=memory_points,
                adaptation_method=adaptation_method, 
                remove_data_bool=remove_data_bool, 
                use_cuda=use_cuda
            )
        
        train_time += time.time() - start_time
            
        train_times.append(train_time)
        
        # test
            
        model.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [12]:
for adaptation_method in adaptation_methods:

    results = list()
    
    for percentage in tqdm(PERCENTAGES):
        
        model = CNN().cuda()
        
        model.load_state_dict(torch.load('./weights/original/005.pt'))
        
        train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
        
        train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/kpriors/{adaptation_method}/{percentage}', train_set, test_set, forget_set)
        
        df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
        df['epoch'] = range(1, EPOCHS+1)
        df['percentage'] = percentage
        
        results.append(df)

    results = pd.concat(results).set_index(['percentage', 'epoch'])
    
    results.to_csv(f'results/kpriors/{adaptation_method}.csv')

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        6.384204   0.970904  0.971246    0.971386
           2        6.487081   0.957694  0.959565    0.960185
           3        6.516479   0.967269  0.967851    0.968373
           4        6.594802   0.971931  0.972943    0.973080
           5        6.265817   0.956617  0.956969    0.957549
10         1       16.795093   0.975544  0.976138    0.975923
           2       16.587360   0.964085  0.966354    0.966567
           3       16.880488   0.966677  0.969649    0.969561
           4       17.011639   0.944887  0.945887    0.947605
           5       16.907945   0.953384  0.954473    0.956649
20         1       20.320035   0.959125  0.960763    0.960074
           2       19.417346   0.954896  0.954772    0.954079
           3       19.539327   0.963417  0.965954    0.963254
           4       19.620408   0.955312  0.957768    0.955532
           5       19.559124   0.948521  0.950779    0.949173
30         1       20.140157   0.899419  0.902556    0.899579
           2       20.996589   0.733958  0.734425    0.733626
           3       21.035202   0.959206  0.959864    0.959511
           4       20.399345   0.910653  0.912640    0.911102
           5       20.747788   0.914985  0.915635    0.911672
40         1       21.183923   0.828944  0.828574    0.830433
           2       21.335480   0.766556  0.765176    0.767168
           3       21.301043   0.903083  0.904453    0.907044
           4       20.913690   0.931222  0.930611    0.930503
           5       21.012361   0.876861  0.878494    0.878939
50         1       21.750778   0.355410  0.352137    0.350195
           2       22.057901   0.859275  0.861122    0.856540
           3       21.852227   0.832922  0.834165    0.831310
           4       22.523047   0.892024  0.892572    0.890338
           5       21.850565   0.757696  0.770966    0.758518
60         1       20.742844   0.814292  0.813898    0.810349
           2       21.556610   0.379375  0.381190    0.378412
           3       21.468966   0.641542  0.652456    0.643428
           4       21.421669   0.865042  0.866014    0.861461
           5       21.288592   0.816833  0.817292    0.812044
70         1       20.846441   0.594694  0.594449    0.587023
           2       21.131107   0.778031  0.766573    0.768796
           3       20.579022   0.448324  0.444289    0.439998
           4       20.767252   0.508825  0.508087    0.503056
           5       20.502395   0.434780  0.428315    0.431581
80         1       19.752287   0.184167  0.183307    0.180312
           2       19.717808   0.258417  0.261282    0.258911
           3       20.207478   0.105667  0.113419    0.113813
           4       21.309026   0.322833  0.325978    0.323997
           5       19.911584   0.445250  0.446685    0.441395
90         1       17.469472   0.525266  0.519369    0.515898
           2       18.320393   0.113863  0.104433    0.105541
           3       18.338756   0.106715  0.114617    0.114318
           4       18.129450   0.206283  0.214557    0.215642
           5       17.954962   0.166390  0.170627    0.165683
99         1        6.548265   0.153509  0.159545    0.157805
           2        6.961410   0.170504  0.166733    0.165726
           3        6.759411   0.118969  0.126098    0.123574
           4        6.917278   0.112939  0.095847    0.098085
           5        6.966128   0.075110  0.091154    0.093361